In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

import sys
import os

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)
    
from utils import create_sliding_windows, SequentialMIONetDataset, SequentialDeepONetDataset
from s_mionet import SequentialMIONet
from s_deeponet import SequentialDeepONet

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [27]:
def init_single_model(b_type: str):
    dim = 128
    model = SequentialDeepONet(
        branch_type=b_type,
        branch_input_size=12,
        branch_hidden_size=128,
        branch_num_layers=4,
        branch_output_size=dim,
        trunk_architecture=[2, 128, 128, dim],
        num_outputs=1,
        use_transform=True,
        activation_fn=nn.ReLU,
    )
    return model

In [28]:
def init_multi_model(b_type: str):
    dim = 128

    # Define a single branch configuration to be reused
    base_branch_config = {
        "type": b_type,
        "input_size": 1,
        "hidden_size": 128,
        "num_layers": 4,
        "output_size": dim
    }

    # Create a dictionary with the same branch configuration for 12 branches
    branches_config = {f"sensor{i+1}": base_branch_config for i in range(12)}

    # Trunk network configuration
    trunk_architecture = [2, 128, 128, dim]
    num_outputs = 1

    # Instantiate the model with the replicated branches
    model = SequentialMIONet(branches_config, trunk_architecture, num_outputs)

    return model

In [29]:
# architecture
arch = ['single_branch', 'multi_branch']

# branch type
b_type = ['lstm', 'gru']

# sequence length
s_length = [7, 30, 60, 90]

In [30]:
def load_model(arch, b_type, s_length, device):
    
    # condition to load the model
    if arch == 'single_branch':
        if b_type == 'lstm':
            model = init_single_model(b_type).to(device)
        else:
            model = init_single_model(b_type).to(device)
    elif arch == 'multi_branch':
        if b_type == 'lstm':
            model = init_multi_model(b_type).to(device)
        else:
            model = init_multi_model(b_type).to(device)
    
    # set the model parameters path
    dir_name = f"{arch}/{b_type}_window_{s_length}.pth"
    print(f"Model path: {dir_name}")
    
    # load the model
    model.load_state_dict(torch.load(dir_name))
    
    return model

In [31]:
# Load neutron monitoring data
input_data = np.load('data/neutron_data_22yrs.npy')
trunk = np.load('data/grid_points.npy')
target = np.load('data/dose_array.npy')

# Normalize trunk input
trunk[:, 0] = (trunk[:, 0] - np.min(trunk[:, 0])) / (np.max(trunk[:, 0]) - np.min(trunk[:, 0]))
trunk[:, 1] = (trunk[:, 1] - np.min(trunk[:, 1])) / (np.max(trunk[:, 1]) - np.min(trunk[:, 1]))

# %%
def train_val_test_split(input_data, target):
    # Define the number of test samples (last 365 days)
    test_size = 365

    # Split data into training+validation and test
    train_val_input = input_data[:-test_size]
    train_val_target = target[:-test_size]
    test_input = input_data[-test_size:]
    test_target = target[-test_size:]

    # Calculate split index for training and validation
    train_size = int(len(train_val_input) * 0.5)  # 80% for training
    val_size = len(train_val_input) - train_size  # 20% for validation

    # Training set
    train_input = train_val_input[:train_size]
    train_target = train_val_target[:train_size]

    # Validation set
    val_input = train_val_input[train_size:]
    val_target = train_val_target[train_size:]

    # Final shapes check
    print("Train input shape:", train_input.shape)
    print("Validation input shape:", val_input.shape)
    print("Test input shape:", test_input.shape)

    return train_input, train_target, val_input, val_target, test_input, test_target

# Assuming input_data and target are defined elsewhere in the notebook
train_input, train_target, val_input, val_target, test_input, test_target = train_val_test_split(input_data, target)


scaler = MinMaxScaler()
train_input = scaler.fit_transform(train_input)
val_input = scaler.transform(val_input)
test_input = scaler.transform(test_input)

# target data normalization (min-max scaling)
scaler_target = MinMaxScaler()
train_target = scaler_target.fit_transform(train_target)[..., np.newaxis]
val_target = scaler_target.transform(val_target)[..., np.newaxis]
test_target = scaler_target.transform(test_target)[..., np.newaxis]

In [32]:
def create_sequence(arch, s_length, input_data, trunk_data, target):
    # Create sliding windows
    test_input_seq, test_target_seq = create_sliding_windows(test_input, test_target, s_length)
    print("Test input shape:", test_input_seq.shape)
    print("Test target shape:", test_target_seq.shape)
    
    # Create dataset and dataloader
    batch_size = 16
    if arch == 'single_branch':
        test_dataset = SequentialDeepONetDataset(test_input_seq, trunk, test_target_seq)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    else:
        test_dataset = SequentialMIONetDataset(test_input_seq, trunk, test_target_seq)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_loader

In [33]:
import time
import torch
import numpy as np

def compute_time(model, test_loader, arch, warmup_iters=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Warm-up iterations to stabilize GPU performance
    with torch.no_grad():
        for _ in range(warmup_iters):
            for batch in test_loader:
                if arch == 'single_branch':
                    x, trunk, y = batch
                    x, trunk, y = x.to(device), trunk.to(device), y.to(device)
                    _ = model(x, trunk)
                else:  # multi_branch case
                    branch_batch, trunk_batch, target_batch = batch
                    if isinstance(branch_batch, dict):
                        branch_batch = {key: value.to(device) for key, value in branch_batch.items()}
                    else:
                        branch_batch = branch_batch.to(device)
                    trunk_batch = trunk_batch.to(device)
                    target_batch = target_batch.to(device)
                    _ = model(branch_batch, trunk_batch)

    # Collect inference times per batch
    batch_times = []
    sample_counts = []

    with torch.no_grad():
        for batch in test_loader:
            batch_size = len(batch[0])  # Assuming batch[0] holds the input tensor
            sample_counts.append(batch_size)  # Track sample count
            
            torch.cuda.synchronize()
            start_time = time.time()
            
            if arch == 'single_branch':
                x, trunk, y = batch
                x, trunk, y = x.to(device), trunk.to(device), y.to(device)
                _ = model(x, trunk)
            else:
                branch_batch, trunk_batch, target_batch = batch
                if isinstance(branch_batch, dict):
                    branch_batch = {key: value.to(device) for key, value in branch_batch.items()}
                else:
                    branch_batch = branch_batch.to(device)
                trunk_batch = trunk_batch.to(device)
                target_batch = target_batch.to(device)
                _ = model(branch_batch, trunk_batch)

            torch.cuda.synchronize()
            end_time = time.time()
            batch_times.append(end_time - start_time)

    # Compute statistics
    total_samples = sum(sample_counts)
    mean_time = np.mean(batch_times) / np.mean(sample_counts) if total_samples > 0 else float('inf')
    std_time = np.std(batch_times) / np.mean(sample_counts) if total_samples > 0 else float('inf')

    # Print results in scientific notation
    print(f"Model: {arch}")
    print(f"Mean Inference Time per Sample: {mean_time:.3e} ± {std_time:.3e} seconds/sample\n")


In [34]:
import sys
import torch

# Define the output log file path
log_file_path = "temp/inference_time_log.txt"

# Open the file and redirect print statements
with open(log_file_path, "w") as f:
    sys.stdout = f  # Redirect print statements to file
    
    # Compute inference time for all models and configurations
    for a in arch:
        for b in b_type:
            for s in s_length:
                model = load_model(a, b, s, device).eval()
                test_loader = create_sequence(a, s, test_input, trunk, test_target)

                # Compute trainable parameters
                num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

                print(f"Model: {a}, Branch type: {b}, Sequence length: {s}")
                print(f"Trainable Parameters: {num_params:,}")  # Format with comma separator

                compute_time(model, test_loader, a)
                print("\n")

# Restore standard output back to normal
sys.stdout = sys.__stdout__

print(f"Log saved to {log_file_path}")
